In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization, MaxPool2D
from IPython.display import Image, display
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import *

In [2]:
len(tf.config.list_physical_devices('GPU'))

1

In [7]:
def import_maps(path):
    print(path)
    images = []
    for index, name in enumerate(os.listdir(path)):
        folder = os.path.join(path, name)
        for file_class in os.listdir(folder):
            #if 'patches' in file_class:
            im_folder = os.path.join(folder, file_class)
#                 print(im_folder)
#                 break
            for im in os.listdir(im_folder):
                img = cv2.imread(os.path.join(im_folder, im))
                #print(img.shape)
                if img is not None:
                    img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
                    img1 = cv2.resize(img, (64, 64))
                    img = np.dstack((img1,img1,img1))
                if img is not None:
               #     img = (img-np.mean(img))/np.std(img)
                    images.append((np.array(img), index)) 
    return images

In [8]:
image_train_set = import_maps(r'C:\Users\AIIMS-IITD\Desktop\Sanjeev\4_class_cluster_volume\Train')
image_test_set = import_maps(r'C:\Users\AIIMS-IITD\Desktop\Sanjeev\4_class_cluster_volume\Test')

C:\Users\AIIMS-IITD\Desktop\Sanjeev\4_class_cluster_volume\Train
C:\Users\AIIMS-IITD\Desktop\Sanjeev\4_class_cluster_volume\Test


In [9]:
len(image_train_set)

17155

In [10]:
len(image_test_set)

2160

In [11]:
train_images_all = [i[0] for i in image_train_set]
train_images_array = np.array(train_images_all)
#train_images_array=np.expand_dims(train_images_array,axis=3)
train_image_label = [i[1] for i in image_train_set]
train_image_label = np.array(train_image_label)

In [12]:
test_images_all = [i[0] for i in image_test_set]
test_images_array = np.array(test_images_all)
#test_images_array=np.expand_dims(test_images_array,axis=3)
test_image_label = [i[1] for i in image_test_set]
test_image_label = np.array(test_image_label)

In [13]:
scalar = StandardScaler(copy=False)
train_images = scalar.fit_transform(train_images_array.reshape(len(train_images_array), 64*64*3))
train_images_array = train_images.reshape(len(train_images_array), 64,64, 3)

In [14]:
test_image = scalar.fit_transform(test_images_array.reshape(len(test_images_array),64*64*3))
test_image_array = test_image.reshape(len(test_images_array),64,64, 3)

In [15]:
enc = OneHotEncoder(handle_unknown='ignore')
train_label_enc = enc.fit_transform(train_image_label.reshape(-1, 1)).toarray()
#val_label_enc = enc.fit_transform(val_y.reshape(-1, 1)).toarray()
test_label_enc = enc.fit_transform(test_image_label.reshape(-1, 1)).toarray()

In [23]:
model = Sequential()
model.add(Conv2D(input_shape=(64,64,3),filters=64,kernel_size=(5,5),padding='same', activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(MaxPooling2D(2,2))

model.add(Conv2D(filters=32,kernel_size=(3,3),padding='same', activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.2))



model.add(Conv2D(filters=16,kernel_size=(3,3),padding='same', activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.01)))
model.add(BatchNormalization())
#model.add(MaxPooling2D(2,2))
model.add(Dropout(0.5))

model.add(Flatten(name='flatten'))
#model.add(Dense(16, activation='relu', name='fc1'))
model.add(Dense(4, activation='sigmoid'))

model.compile(loss=keras.losses.categorical_crossentropy,
               optimizer=keras.optimizers.Adam(0.001),
               metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 64, 64, 64)        4864      
_________________________________________________________________
batch_normalization_6 (Batch (None, 64, 64, 64)        256       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 32, 32, 32)        18464     
_________________________________________________________________
batch_normalization_7 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 16, 16, 32)       

In [24]:
history = model.fit(train_images_array,train_label_enc,validation_split=0.2, batch_size=32, epochs=100, verbose=1,shuffle=True)

Epoch 1/100
429/429 [==============================] - 4s 8ms/step - loss: 1.0799 - accuracy: 0.8360 - val_loss: 3.4894 - val_accuracy: 0.1775
Epoch 2/100
429/429 [==============================] - 3s 8ms/step - loss: 0.4979 - accuracy: 0.9669 - val_loss: 5.3632 - val_accuracy: 0.1618
Epoch 3/100
429/429 [==============================] - 3s 8ms/step - loss: 0.3421 - accuracy: 0.9769 - val_loss: 4.9065 - val_accuracy: 0.1723
Epoch 4/100
429/429 [==============================] - 3s 8ms/step - loss: 0.2796 - accuracy: 0.9749 - val_loss: 8.3122 - val_accuracy: 0.0842
Epoch 5/100
429/429 [==============================] - 3s 8ms/step - loss: 0.2420 - accuracy: 0.9788 - val_loss: 5.2726 - val_accuracy: 0.2906
Epoch 6/100
429/429 [==============================] - 3s 8ms/step - loss: 0.2196 - accuracy: 0.9816 - val_loss: 7.4240 - val_accuracy: 0.1035
Epoch 7/100
429/429 [==============================] - 3s 8ms/step - loss: 0.2175 - accuracy: 0.9805 - val_loss: 9.1772 - val_accuracy: 0.1291

Epoch 58/100
429/429 [==============================] - 3s 8ms/step - loss: 0.0829 - accuracy: 0.9946 - val_loss: 10.7937 - val_accuracy: 0.0632
Epoch 59/100
429/429 [==============================] - 3s 8ms/step - loss: 0.0787 - accuracy: 0.9942 - val_loss: 8.8883 - val_accuracy: 0.1449
Epoch 60/100
429/429 [==============================] - 3s 8ms/step - loss: 0.0855 - accuracy: 0.9929 - val_loss: 7.8932 - val_accuracy: 0.1259
Epoch 61/100
429/429 [==============================] - 3s 8ms/step - loss: 0.0875 - accuracy: 0.9938 - val_loss: 11.6069 - val_accuracy: 0.0723
Epoch 62/100
429/429 [==============================] - 3s 8ms/step - loss: 0.0818 - accuracy: 0.9945 - val_loss: 8.0757 - val_accuracy: 0.1140
Epoch 63/100
429/429 [==============================] - 3s 8ms/step - loss: 0.0876 - accuracy: 0.9932 - val_loss: 10.2208 - val_accuracy: 0.0533
Epoch 64/100
429/429 [==============================] - 3s 8ms/step - loss: 0.0892 - accuracy: 0.9932 - val_loss: 7.3027 - val_accura

In [25]:
score = model.evaluate(test_image_array,test_label_enc, batch_size=32)

68/68 [==============================] - 0s 3ms/step - loss: 4.9177 - accuracy: 0.4259


In [26]:
score = model.evaluate(train_images_array,train_label_enc, batch_size=32)

537/537 [==============================] - 2s 3ms/step - loss: 2.0662 - accuracy: 0.8103


# Old

In [29]:
model = Sequential()

model.add(Conv2D(input_shape=(112,112,3),filters=128,kernel_size=(3,3),padding="same", activation='LeakyReLU'))
model.add(Dropout(0.2))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="LeakyReLU"))
model.add(Dropout(0.2))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Dropout(0.2))

model.add(Conv2D(filters=48,kernel_size=(1,1),padding="same", activation="LeakyReLU"))
model.add(Dropout(0.2))

model.add(Conv2D(filters=32,kernel_size=(1,1),padding="same", activation="LeakyReLU"))
model.add(Dropout(0.2))

model.add(Conv2D(filters=24,kernel_size=(1,1),padding="same", activation="LeakyReLU"))
model.add(Dropout(0.2))

model.add(Flatten(name='flatten'))
model.add(Dense(16, activation='LeakyReLU', name='fc1'))
model.add(Dropout(0.3))
model.add(Dense(4, activation='sigmoid', name='fc2'))

model.compile(loss=keras.losses.categorical_crossentropy,
               optimizer=keras.optimizers.Adam(0.0012),
               metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 112, 112, 128)     3584      
_________________________________________________________________
dropout_12 (Dropout)         (None, 112, 112, 128)     0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 112, 112, 64)      73792     
_________________________________________________________________
dropout_13 (Dropout)         (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 112, 112, 64)      36928     
_________________________________________________________________
dropout_14 (Dropout)         (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 112, 112, 48)     

In [30]:
from tensorflow.keras.layers import LeakyReLU


In [31]:
history = model.fit(train_images_array,train_label_enc,validation_data=(test_images_array,test_label_enc), batch_size=32, epochs=5, verbose=1,shuffle=True)

Epoch 1/5
239/239 [==============================] - 19s 77ms/step - loss: 0.4049 - accuracy: 0.8607 - val_loss: 328.5982 - val_accuracy: 0.3108
Epoch 2/5
239/239 [==============================] - 18s 77ms/step - loss: 0.0630 - accuracy: 0.9820 - val_loss: 359.2047 - val_accuracy: 0.3378
Epoch 3/5
239/239 [==============================] - 18s 77ms/step - loss: 0.0249 - accuracy: 0.9923 - val_loss: 403.9989 - val_accuracy: 0.3099
Epoch 4/5
239/239 [==============================] - 18s 77ms/step - loss: 0.0528 - accuracy: 0.9866 - val_loss: 279.6661 - val_accuracy: 0.3432
Epoch 5/5
239/239 [==============================] - 18s 77ms/step - loss: 0.0682 - accuracy: 0.9833 - val_loss: 336.4004 - val_accuracy: 0.2108


In [32]:
score = model.evaluate(test_image_array,test_label_enc, batch_size=16)

70/70 [==============================] - 1s 11ms/step - loss: 9.9756 - accuracy: 0.1937
